# Unit Testing for NRPy+

## Author: Kevin Lituchy

## This module sets up the unit testing infrastructure required to begin unit testing any given NRPy+ module. 

### NRPy+ Source Code for this module: 
* [UnitTesting/example/module1.py](../../edit/UnitTesting/example/module1.py)
* [UnitTesting/example/module2.py](../../edit/UnitTesting/example/module2.py)
* [UnitTesting/example/trusted_values_dict.py](../../edit/UnitTesting/example/trusted_values_dict.py)
* [UnitTesting/example/NRPyUnitTests_example_Globals.py](../../edit/UnitTesting/example/NRPyUnitTests_example_Globals.py)

## Introduction

What is the purpose of unit testing, and why should you do it? To begin thinking about that, consider what subtleties 
can occur with your code that are almost unnoticeable to the eye, but wind up giving you a very incorrect result. You
could make a small optimization, and observe that nothing changes about your result. However, maybe the optimization 
you made only works on Python 3 and not Python 2, or it changes a value by some tiny amount -- too small to be obviously
noticeable, but enough to make a difference.

This is where unit testing comes in. By initially calculating values for the globals of your modules in a **trusted**
version of your code and storing those values in a dictionary, you can then easily check if something stopped working
by comparing your newly calculated values to the ones you've stored. On the frontend, there are four modules 
essential to get your unit tests up and running: `trusted_values_dict`, `functions_and_globals`, `run_test`, and your 
testing modules (which we'll simply reference as `module1` and `module2`). The usage of each of these modules is outlined in 
the  **Interactive Modules** section. There is also some important prerequisite knowledge that may be helpful to grasp
before beginning your testing. There are many functions at play in the backend as well, all of which will 
be described in detail below in the **Functions** section. Mastery of these functions may not be essential to get your 
tests up-and-running, but some basic understanding of these modules with undoubtedly streamline the testing process and 
how to potentially create your own, different types of tests.

An important caveat is that the unit testing does not test the **correctness** of your code or your variables. The 
unit tests function as a protective measure to ensure that nothing was broken; it gets its values by running _your_ 
code, so if something starts out incorrect, it will be stored as incorrect in the system. There are measures against 
this, but it relies on the user's knowledge of what versions of their code are correct.


<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

1. [Step 1](#step1): Configure essential files
    1. [Step 1.a](#trusted_values_dict): Creating and formatting trusted_values_dict.py
    1. [Step 1.b](#trusted_values_dict): Creating and formatting NRPyUnitTests_example_Globals.py
1. [Step 2](#step2): Scalar Wave RHSs in One Spatial Dimension, Fourth-Order Finite Differencing
1. [Step 3](#step3): Scalar Wave RHSs in Three Spatial Dimensions, Tenth-Order Finite Differencing
    1. [Step 3.a](#code_validation1): Code Validation against ScalarWave.ScalarWave_RHSs NRPy+ module
1. [Step 4](#step4): Plane-Wave Initial Data for the Scalar Wave Equation
    1. [Step 4.a](#code_validation2): Code Validation against ScalarWave.InitialData_PlaneWave NRPy+ module
1. [Step 5](#latex_pdf_output): Output this module to $\LaTeX$-formatted PDF file

<a id='step1'></a>

# Step 1: Configure essential files \[Back to [top](#toc)\]
$$\label{step1}$$

<a id='trusted_values_dict'></a>

## Step 1.a: Creating and formatting trusted_values_dict.py \[Back to [top](#toc)\]
$$\label{trusted_values_dict}$$

We always begin trusted_values_dict.py as follows:

In [ ]:
# Step P0: Import necessary modules
from mpmath import mpf, mp, mpc
from UnitTesting.standard_constants import precision

# Step 0: Set precision value and initialize trusted_values_dict
mp.dps = precision
trusted_values_dict = dict()

This is all we need to do for now; we'll put the actual trusted values that our module generates in trusted_values_dict.py once we generate them.

<a id='nrpyunittests'></a>

## Step 1.b: Creating and formatting NRPyUnitTests_example_Globals.py \[Back to [top](#toc)\]
$$\label{nrpyunittests}$$

<a id='step2'></a>

# Step 2: Scalar Wave RHSs in One Spatial Dimension, Fourth-Order Finite Differencing \[Back to [top](#toc)\]
$$\label{step2}$$

To minimize complication, we will first restrict ourselves to solving the wave equation in one spatial dimension, so
$$\nabla^2 u = \partial_x^2 u.$$
Extension of this operator to higher spatial dimensions is straightforward, particularly when using NRPy+.

As was discussed in [the finite difference section of the tutorial](Tutorial-Finite_Difference_Derivatives.ipynb), NRPy+ approximates derivatives using [finite difference methods](),  the second-order derivative $\partial_x^2$ accurate to fourth-order in uniform grid spacing $\Delta x$ (from fitting the unique 4th-degree polynomial to 5 sample points of $u$) is given by
\begin{equation}
\left[\partial_x^2 u(t,x)\right]_j = \frac{1}{(\Delta x)^2}
\left(
-\frac{1}{12} \left(u_{j+2} + u_{j-2}\right) 
+ \frac{4}{3}  \left(u_{j+1} + u_{j-1}\right)
- \frac{5}{2} u_j \right)
+ \mathcal{O}\left((\Delta x)^4\right).
\end{equation}

In [2]:
# Step P2: Define the C parameter wavespeed. The `wavespeed`
#          variable is a proper SymPy variable, so it can be
#          used in below expressions. In the C code, it acts
#          just like a usual parameter, whose value is 
#          specified in the parameter file.
thismodule = "ScalarWave"
wavespeed = par.Cparameters("REAL",thismodule,"wavespeed")

# Step 1: Set the spatial dimension parameter, and then read
#         the parameter as DIM.
par.set_parval_from_str("grid::DIM",1)
DIM = par.parval_from_str("grid::DIM")

# Step 2: Set the finite differencing order to 4.
par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER",4)

# Step 3: Register gridfunctions that are needed as input
#         to the scalar wave RHS expressions.
uu, vv = gri.register_gridfunctions("EVOL",["uu","vv"])

# Step 4: Declare the rank-2 indexed expression \partial_{ij} u,
#         which is symmetric about interchange of indices i and j
#         Derivative variables like these must have an underscore
#         in them, so the finite difference module can parse the
#         variable name properly.
uu_dDD = ixp.declarerank2("uu_dDD","sym01")

# Step 5: Define right-hand sides for the evolution.
uu_rhs = vv
vv_rhs = 0
for i in range(DIM):
    vv_rhs += wavespeed*wavespeed*uu_dDD[i][i]

vv_rhs = sp.simplify(vv_rhs)

# Step 6: Generate C code for scalarwave evolution equations,
#         print output to the screen (standard out, or stdout).
fin.FD_outputC("stdout",
               [lhrh(lhs=gri.gfaccess("rhs_gfs","uu"),rhs=uu_rhs),
                lhrh(lhs=gri.gfaccess("rhs_gfs","vv"),rhs=vv_rhs)])

{
   /* 
    * NRPy+ Finite Difference Code Generation, Step 1 of 2: Read from main memory and compute finite difference stencils:
    */
   /*
    *  Original SymPy expression:
    *  "const double uu_dDD00 = invdx0**2*(-5*uu/2 + 4*uu_i0m1/3 - uu_i0m2/12 + 4*uu_i0p1/3 - uu_i0p2/12)"
    */
   const double uu_i0m2 = in_gfs[IDX2(UUGF, i0-2)];
   const double uu_i0m1 = in_gfs[IDX2(UUGF, i0-1)];
   const double uu = in_gfs[IDX2(UUGF, i0)];
   const double uu_i0p1 = in_gfs[IDX2(UUGF, i0+1)];
   const double uu_i0p2 = in_gfs[IDX2(UUGF, i0+2)];
   const double vv = in_gfs[IDX2(VVGF, i0)];
   const double uu_dDD00 = pow(invdx0, 2)*(-5.0/2.0*uu + (4.0/3.0)*uu_i0m1 - 1.0/12.0*uu_i0m2 + (4.0/3.0)*uu_i0p1 - 1.0/12.0*uu_i0p2);
   /* 
    * NRPy+ Finite Difference Code Generation, Step 2 of 2: Evaluate SymPy expressions and write to main memory:
    */
   /*
    *  Original SymPy expressions:
    *  "[rhs_gfs[IDX2(UUGF, i0)] = vv,
    *    rhs_gfs[IDX2(VVGF, i0)] = uu_dDD00*wavespeed**2]"
    */
  

**Success!** Notice that indeed NRPy+ was able to compute the spatial derivative operator,
\begin{equation}
\left[\partial_x^2 u(t,x)\right]_j \approx \frac{1}{(\Delta x)^2}
\left(
-\frac{1}{12} \left(u_{j+2} + u_{j-2}\right) 
+ \frac{4}{3}  \left(u_{j+1} + u_{j-1}\right)
- \frac{5}{2} u_j \right),
\end{equation}
correctly (easier to read in the "Original SymPy expressions" comment block at the top of the C output. Note that $\texttt{invdx0}=1/\Delta x_0$, where $\Delta x_0$ is the (uniform) grid spacing in the zeroth, or $x_0$ direction.

<a id='step3'></a>

# Step 3: Scalar Wave RHSs in Three Spatial Dimensions, Tenth-Order Finite Differencing \[Back to [top](#toc)\]
$$\label{step3}$$

Let's next repeat the same process, only this time at **10th** finite difference order, for the **3-spatial-dimension** scalar wave equation, with SIMD enabled:

In [3]:
# Step 1: Define the C parameter wavespeed. The `wavespeed`
#         variable is a proper SymPy variable, so it can be
#         used in below expressions. In the C code, it acts
#         just like a usual parameter, whose value is 
#         specified in the parameter file.
wavespeed = par.Cparameters("REAL",thismodule,"wavespeed")

# Step 2: Set the spatial dimension parameter 
#         to *FOUR* this time, and then read
#         the parameter as DIM.
par.set_parval_from_str("grid::DIM",3)
DIM = par.parval_from_str("grid::DIM")

# Step 3: Set the finite differencing order to 10.
par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER",10)

# Step 4a: Reset gridfunctions registered in 1D case above,
#          to avoid NRPy+ throwing an error about double-
#          registering gridfunctions, which is not allowed.
gri.glb_gridfcs_list = []

# Step 4b: Register gridfunctions that are needed as input
#          to the scalar wave RHS expressions.
uu, vv = gri.register_gridfunctions("EVOL",["uu","vv"])

# Step 5: Declare the rank-2 indexed expression \partial_{ij} u,
#         which is symmetric about interchange of indices i and j
#         Derivative variables like these must have an underscore
#         in them, so the finite difference module can parse the
#         variable name properly.
uu_dDD = ixp.declarerank2("uu_dDD","sym01")

# Step 6: Define right-hand sides for the evolution.
uu_rhs = vv
vv_rhs = 0
for i in range(DIM):
    vv_rhs += wavespeed*wavespeed*uu_dDD[i][i]

# Step 7: Simplify the expression for c^2 \nabla^2 u (a.k.a., vv_rhs):
vv_rhs = sp.simplify(vv_rhs)

# Step 8: Generate C code for scalarwave evolution equations,
#         print output to the screen (standard out, or stdout).
fin.FD_outputC("stdout",
               [lhrh(lhs=gri.gfaccess("rhs_gfs","uu"),rhs=uu_rhs),
                lhrh(lhs=gri.gfaccess("rhs_gfs","vv"),rhs=vv_rhs)],params="SIMD_enable=True")

initialize_param() minor warning: Did nothing; already initialized parameter ScalarWave::wavespeed
{
   /* 
    * NRPy+ Finite Difference Code Generation, Step 1 of 2: Read from main memory and compute finite difference stencils:
    */
   /*
    *  Original SymPy expressions:
    *  "[const REAL_SIMD_ARRAY uu_dDD00 = invdx0**2*(-5269*uu/1800 + 5*uu_i0m1_i1_i2/3 - 5*uu_i0m2_i1_i2/21 + 5*uu_i0m3_i1_i2/126 - 5*uu_i0m4_i1_i2/1008 + uu_i0m5_i1_i2/3150 + 5*uu_i0p1_i1_i2/3 - 5*uu_i0p2_i1_i2/21 + 5*uu_i0p3_i1_i2/126 - 5*uu_i0p4_i1_i2/1008 + uu_i0p5_i1_i2/3150),
    *    const REAL_SIMD_ARRAY uu_dDD11 = invdx1**2*(-5269*uu/1800 + 5*uu_i0_i1m1_i2/3 - 5*uu_i0_i1m2_i2/21 + 5*uu_i0_i1m3_i2/126 - 5*uu_i0_i1m4_i2/1008 + uu_i0_i1m5_i2/3150 + 5*uu_i0_i1p1_i2/3 - 5*uu_i0_i1p2_i2/21 + 5*uu_i0_i1p3_i2/126 - 5*uu_i0_i1p4_i2/1008 + uu_i0_i1p5_i2/3150),
    *    const REAL_SIMD_ARRAY uu_dDD22 = invdx2**2*(-5269*uu/1800 + 5*uu_i0_i1_i2m1/3 - 5*uu_i0_i1_i2m2/21 + 5*uu_i0_i1_i2m3/126 - 5*uu_i0_i1_i2m4/1008 + u

<a id='code_validation1'></a>

## Step 3.a:  Code Validation against ScalarWave.ScalarWave_RHSs NRPy+ module \[Back to [top](#toc)\]
$$\label{code_validation1}$$

Here, as a code validation check, we verify agreement in the SymPy expressions for the RHSs of the three-spatial-dimension Scalar Wave equation (i.e., uu_rhs and vv_rhs) between

1. this tutorial and 
2. the [NRPy+ ScalarWave.ScalarWave_RHSs](../edit/ScalarWave/ScalarWave_RHSs.py) module.

In [4]:
# Step 10: We already have SymPy expressions for uu_rhs and vv_rhs in
#          terms of other SymPy variables. Even if we reset the list
#          of NRPy+ gridfunctions, these *SymPy* expressions for
#          uu_rhs and vv_rhs *will remain unaffected*. 
# 
#          Here, we will use the above-defined uu_rhs and vv_rhs to 
#          validate against the same expressions in the 
#          ScalarWave/ScalarWave_RHSs.py module,
#          to ensure consistency between this tutorial 
#          (historically speaking, the tutorial was written first) 
#          and the ScalarWave_RHSs.py module itself.
#
# Reset the list of gridfunctions, as registering a gridfunction
#   twice will spawn an error.
gri.glb_gridfcs_list = []

# Step 11: Call the ScalarWave_RHSs() function from within the
#         ScalarWave/ScalarWave_RHSs.py module,
#         which should do exactly the same as in Steps 1-10 above.
import ScalarWave.ScalarWave_RHSs as swrhs
swrhs.ScalarWave_RHSs()

# Step 12: Consistency check between the tutorial module above
#         and the ScalarWave_RHSs() function from within the
#         ScalarWave/ScalarWave_RHSs.py module.
# It is SAFE to ignore the warning from re-initializing the parameter RMAX.
print("^^^ Ignore the minor warning above. ^^^\n")
print("Consistency check between ScalarWave tutorial and NRPy+ module:")
print("uu_rhs - swrhs.uu_rhs: Should be zero: ",sp.simplify(uu_rhs - swrhs.uu_rhs))
print("vv_rhs - swrhs.vv_rhs: Should be zero: ",sp.simplify(vv_rhs - swrhs.vv_rhs))

^^^ Ignore the minor warning above. ^^^

Consistency check between ScalarWave tutorial and NRPy+ module:
uu_rhs - swrhs.uu_rhs: Should be zero:  0
vv_rhs - swrhs.vv_rhs: Should be zero:  0


<a id='step4'></a>

# Step 4: Plane-Wave Initial Data for the Scalar Wave Equation \[Back to [top](#toc)\]
$$\label{step4}$$

The solution to the scalar wave equation for a monochromatic (single-wavelength) wave traveling in the $\hat{k}$ direction is
$$u(\vec{x},t) = f(\hat{k}\cdot\vec{x} - c t),$$
where $\hat{k}$ is a unit vector. We choose $f(\hat{k}\cdot\vec{x} - c t)$ to take the form
$$
f(\hat{k}\cdot\vec{x} - c t) = \sin\left(\hat{k}\cdot\vec{x} - c t\right) + 2,
$$
where we add the $+2$ to ensure that the exact solution never crosses through zero. In places where the exact solution passes through zero, the relative error (i.e., the measure of error to compare numerical with exact results) is undefined. Also, $f(\hat{k}\cdot\vec{x} - c t)$ plus a constant is still a solution to the wave equation.

In [5]:
# Step 1: Set parameters defined in other modules
xx = gri.xx

# Step 2: Declare free parameters intrinsic to these initial data
time = par.Cparameters("REAL",thismodule,"time")
kk   = par.Cparameters("REAL",thismodule,["kk0","kk1","kk2"])

# Step 3: Normalize the k vector
kk_norm = sp.sqrt(kk[0]**2 + kk[1]**2 + kk[2]**2)

# Step 4: Compute k.x
dot_product = sp.sympify(0)
for i in range(DIM):
    dot_product += xx[i]*kk[i]
dot_product /= kk_norm

# Step 5: Set initial data for uu and vv, where vv_ID = \partial_t uu_ID.
uu_ID = sp.sin(dot_product - wavespeed*time)+2
vv_ID = sp.diff(uu_ID, time)

Next we verify that $f(\hat{k}\cdot\vec{x} - c t)$ satisfies the wave equation, by computing
$$\left(c^2 \nabla^2 - \partial_t^2 \right)\ f\left(\hat{k}\cdot\vec{x} - c t\right),$$
and confirming the result is exactly zero.

In [6]:
sp.simplify(wavespeed**2*(sp.diff(uu_ID,xx[0],2) + 
                          sp.diff(uu_ID,xx[1],2) + 
                          sp.diff(uu_ID,xx[2],2))
            - sp.diff(uu_ID,time,2))

0

<a id='code_validation2'></a>

## Step 4.a: Code Validation against ScalarWave.InitialData_PlaneWave NRPy+ module  \[Back to [top](#toc)\]
$$\label{code_validation2}$$

As a code validation check, we will verify agreement in the SymPy expressions for plane-wave initial data for the Scalar Wave equation between
1. this tutorial and 
2. the NRPy+ [ScalarWave.InitialData_PlaneWave](../edit/ScalarWave/InitialData_PlaneWave.py) module.

In [7]:
# We just defined SymPy expressions for uu_ID and vv_ID in
# terms of other SymPy variables. Here, we will use the 
# above-defined uu_ID and vv_ID to validate against the 
# same expressions in the ScalarWave/InitialData_PlaneWave.py 
# module, to ensure consistency between this tutorial 
# (historically speaking, the tutorial was written first) 
# and the PlaneWave ID module itself.
# 
# Step 6: Call the InitialData_PlaneWave() function from within the
#         ScalarWave/InitialData_PlaneWave.py module,
#         which should do exactly the same as in Steps 1-5 above.
import ScalarWave.InitialData_PlaneWave as swid
swid.InitialData_PlaneWave()

# Step 7: Consistency check between the tutorial module above
#         and the InitialData_PlaneWave() function from within the
#         ScalarWave/InitialData_PlaneWave.py module.
# It is SAFE to ignore the warning from re-initializing the parameter RMAX.
print("Consistency check between ScalarWave tutorial and NRPy+ module:")
print("uu_ID - swid.uu_ID: Should be zero: ",sp.simplify(uu_ID - swid.uu_ID))
print("vv_ID - swid.vv_ID: Should be zero: ",sp.simplify(vv_ID - swid.vv_ID))

Consistency check between ScalarWave tutorial and NRPy+ module:
uu_ID - swid.uu_ID: Should be zero:  0
vv_ID - swid.vv_ID: Should be zero:  0


<a id='latex_pdf_output'></a>

# Step 5: Output this module to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-ScalarWave.pdf](Tutorial-ScalarWave.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [8]:
!jupyter nbconvert --to latex --template latex_nrpy_style.tplx Tutorial-ScalarWave.ipynb
!pdflatex -interaction=batchmode Tutorial-ScalarWave.tex
!pdflatex -interaction=batchmode Tutorial-ScalarWave.tex
!pdflatex -interaction=batchmode Tutorial-ScalarWave.tex
!rm -f Tut*.out Tut*.aux Tut*.log

[NbConvertApp] Converting notebook Tutorial-ScalarWave.ipynb to latex
[NbConvertApp] Writing 63651 bytes to Tutorial-ScalarWave.tex
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
